In [1]:
#from veneer.manage import create_command_line

#import veneer-py

from veneer.manage import start, create_command_line, kill_all_now
import veneer
#import geopandas
import pandas as pd

In [2]:
veneer_install = 'C:/PythonScriptingForGBR/VeneerPlugin/v411'
source_version = '4.1.1'
cmd_directory = 'C:\\tempveneer\\veneer_cmd'
path = create_command_line(veneer_install,source_version,dest=cmd_directory)
path

'C:\\tempveneer\\veneer_cmd\\FlowMatters.Source.VeneerCmd.exe'

In [5]:
catchment_project='C:\\Burdekin\\BU_RC8_BL.rsproj'
num_copies=1    # Important - set this to be a number ~ the number of CPU cores in your system!
first_port=9950

In [6]:
processes,ports = start(catchment_project,
                        n_instances=num_copies,
                        ports=first_port,
                        debug=True,
                        veneer_exe=path,
                        remote=False)

Starting C:\tempveneer\veneer_cmd\FlowMatters.Source.VeneerCmd.exe -p 9950 -s  "C:\Burdekin\BU_RC8_BL.rsproj"
[0] Loading plugins

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\DERMTools.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\Dynamic_SedNet.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\GBR_DynSed_Extension.dll

[0] Plugins loaded (3/3)

[0] Opening project file: C:\Burdekin\BU_RC8_BL.rsproj

ERROR[0] log4net:ERROR Failed to find configuration section 'log4net' in the application's .config file. Check your .config file for the <log4net> and <configSections> elements. The configuration section should look like: <section name="log4net" type="log4net.Config.Log4NetConfigurationSectionHandler,log4net" />

[0] Loading project

[0] Project Loaded

[0] Burdekin Rebuild 2014

[0] Burdekin Rebuild 2014

[0] [1:28:06 PM] Veneer, by Flow Matters: http://www.flowmatters.com.au

[0] [1:28:06 PM] Started Source RESTful Service on port:9950

In [7]:
#%matplotlib inline
vs = [veneer.Veneer(port=p) for p in ports]
v = vs[0]
the_network = v.network()


allCatchs = the_network['features'].find_by_feature_type('catchment')
#print(len(allCatchs))

#This dictionary object will hopefully hold everything we need
cols = ['SubCat', 'AreaM2', 'Link', 'LinkUpNode', 'LinkDownNode']
networkDetails = pd.DataFrame(columns=cols)#index=len(allCatchs),

intCount = 0
for thisCat in allCatchs:
    
    catName = thisCat['properties']['name']
    catArea = thisCat['properties']['areaInSquareMeters']
    
    linkID = thisCat['properties']['link']
    thisLink = the_network['features'].find_by_id(linkID)
    linkName = thisLink[0]['properties']['name']
    
    fNodeID = thisLink[0]['properties']['from_node']
    tNodeID = thisLink[0]['properties']['to_node']
    
    fNode = the_network['features'].find_by_id(fNodeID)
    tNode = the_network['features'].find_by_id(tNodeID)
    
    fNodeName = fNode[0]['properties']['name']
    tNodeName = tNode[0]['properties']['name']
    
    networkDetails.loc[intCount] = [catName, catArea, linkName, fNodeName, tNodeName]
    #newRow = [catName, catArea, linkName, fNodeName, tNodeName]
    #networkDetails.append([catName, catArea, linkName, fNodeName, tNodeName])
    
    intCount += 1





In [8]:
networkDetails

,SubCat,AreaM2,Link,LinkUpNode,LinkDownNode
0,SC #1608,311678100.0,link for catchment SC #1608,Node on catchment SC #1608,Node on catchment SC #1604
1,SC #1618,72501300.0,link for catchment SC #1618,Node on catchment SC #1618,Node on catchment SC #1614
2,SC #1628,405900.0,link for catchment SC #1628,Node on catchment SC #1628,Outlet Node25
3,SC #1638,69194700.0,link for catchment SC #1638,Node on catchment SC #1638,Node on catchment SC #1636
4,SC #1648,39296700.0,link for catchment SC #1648,121001a,Node on catchment SC #1647
5,SC #1658,170658000.0,link for catchment SC #1658,Node on catchment SC #1658,Node on catchment SC #1656
6,SC #1668,126879300.0,link for catchment SC #1668,Node on catchment SC #1668,120106b
7,SC #1312,79556400.0,link for catchment SC #1312,Node on catchment SC #1312,Node on catchment SC #1311
8,SC #1009,1489500.0,link for catchment SC #1009,Node on catchment SC #1009,Node on catchment SC #1003
9,SC #655,276474600.0,link for catchment SC #655,Node on catchment SC #655,Node on catchment SC #654


In [9]:
networkDetails.to_csv('C:/Burdekin/BURC8Network.csv', index=False)

In [27]:
kill_all_now(processes)